In [1]:
%pylab --no-import-all
%matplotlib inline
import seaborn as sns
import sys
import pandas as pd
from ipywidgets import widgets
from IPython.display import Markdown, display, clear_output, Audio
import os
from os.path import join, isdir
from glob import iglob
import json
from pprint import pformat
import utils
import pickle
import io
import base64
from scipy.io import wavfile
import birdsonganalysis as bsa

sys.path.append('../model')

from song_model import SongModel

sns.set_palette('colorblind')

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
gridpath = '../model/res/diversity_weight//'
run_paths = sorted([run_path for run_path in iglob(join(gridpath, '*')) if isdir(run_path)])

gridanalysis = utils.GridAnalyser(run_paths, figsize=(10, 10/2.3))

In [3]:
with open(join(gridpath, 'desc.md')) as f:
    display(Markdown(f.read()))

# Run description

## Goals

-

## What have changed compared to previous runs?

-

## Expected Results (Success condition)

-

<!-- ## Notes -->


In [4]:
sns.set_context('talk')

vbox = widgets.VBox()

out = widgets.HTML()

tbs = []
for options in gridanalysis.options_list:
    tb = widgets.ToggleButtons()
    tb.options = sorted(list(options))
    display(tb)
    tbs.append(tb)

def find_id(value):
    out.value = 'looking'
    v = 'Not Found'
    for i in range(len(gridanalysis.run_paths)):
        if gridanalysis.conf[i]['name'] == '+'.join([tb.value for tb in tbs]):
            slider.value = i
            v = 'Found'
            break
    out.value = v
slider = widgets.IntSlider(min=0, max=len(run_paths)-1)
slider.observe(lambda x: gridanalysis.show(x['new'], vbox), names='value')
display(out)
display(slider)
display(vbox)

gridanalysis.show(slider.value, vbox)

for tb in tbs:
    tb.observe(lambda x: find_id(x['new']), names='value')

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


boari score 232.432893137
boari score 232.432893137
boari score 232.432893137
boari score 232.432893137
boari score 232.432893137


In [5]:
display(slider)

Widget Javascript not detected.  It may not be installed or enabled properly.


In [6]:
sns.set_context('talk')